In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [3]:
path = '../raw_data/train'

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=90,
    validation_split = 0.2)

In [5]:
valid_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2)

In [6]:
train_generator = train_datagen.flow_from_directory(
    path,
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training',
    seed = 123)

Found 3460 images belonging to 16 classes.


In [7]:
val_generator = valid_datagen.flow_from_directory(
    path, # same directory as training data
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    seed = 123)

Found 857 images belonging to 16 classes.


In [8]:
class_weight = {0:4. ,
                1:2., 
                2:1.5 ,
                3:2.,
                4:3.,
                5:2.,
                6:1.5,
                7:4.,
                8:2.,
                9:3.,
                10:1.,
                11:4.,
                12:3.,
                13:2.,
                14:3.,
                15:2.
               }

In [20]:
model = VGG16(weights="imagenet", include_top=False, input_shape=(256,256,3))
# Set the first layers to be untrainable
model.trainable = False
# Add layers to the mdoel
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(100, activation='relu')
prediction_layer = layers.Dense(16, activation='softmax')

model = Sequential([model,
                    flatten_layer,
                    dense_layer,
                    prediction_layer])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               3276900   
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1616      
Total params: 17,993,204
Trainable params: 3,278,516
Non-trainable params: 14,714,688
_________________________________________________________________


In [24]:
model.compile(optimizer=optimizers.SGD(learning_rate = 0.1),
          loss=SparseCategoricalCrossentropy(from_logits=True),
          metrics=['accuracy'])

In [22]:
callbacks = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min')

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = val_generator,
    validation_steps = val_generator.samples // 32,
    epochs = 10,
    callbacks = [callbacks])

Epoch 1/10
 49/108 [============>.................] - ETA: 9:15 - loss: 2.7259 - accuracy: 0.1492

In [ ]:
model.save(r'../model/josh_vgg_1')